The data we will be working with in this project comes from: https://sbcb.inf.ufrgs.br/cumida
As the Structural Bioinformatics and Computational Biology Lab puts it, they have created a Curated Microarray Database (CuMiDa) for the analysis of gene expression in cancerous breast tissue.

A microarray is a very useful tool in determining which genes are activated or repressed when certain types of cancers are present. We start with a chip containing genes at known locations on the chip. When a microarray is done, samples of cancerous and healty tissues are taken and the DNA is isolated. Then the mRNA is extracted and red or green cDNA is created. These are then mixed and places on the chip. Green and red cDNA binds to the chip if the complementary DNA is present on the chip. This chip is put into a scanner and then each spot (each gene) is assigned a numerical value depending on the color present on that spot. 

In [55]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
import psycopg2


# Load your data frame (replace 'df' with your actual data frame)
df = pd.read_csv('Breast_GSE45827.csv')

# PostgreSQL connection parameters
db_params = {
    "dbname": "madikloberdanz",
    "user": "madikloberdanz",
    "password": "3141",
    "host": "localhost",
    "port": "5433"
}

# Create a SQLAlchemy engine to connect to the PostgreSQL database
engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}")

# Specify the number of columns you want to sample (e.g., 1000)
num_columns_to_sample = 1000

# Check if the table already exists in the database
table_name = "breast_cancer"
inspector = inspect(engine)

if not inspector.has_table(table_name):
  # Get the list of all column names excluding the first two columns
  all_columns = df.columns[1:]

  # Randomly sample column names (excluding the first column)
  sampled_columns = np.random.choice(all_columns, num_columns_to_sample, replace=False)

  # Include the first column in the sampled columns
  sampled_columns = np.insert(sampled_columns, 0, df.columns[1])

  # Create a new data frame with the sampled columns
  sampled_df = df[sampled_columns]

  # Use SQLAlchemy to write the data frame to a PostgreSQL table
  sampled_df.to_sql(table_name, engine, schema='public', index=False, if_exists='replace')


# Dispose of the engine (optional but good practice)
engine.dispose()

import psycopg2


# Establish a connection to the PostgreSQL database
connection = psycopg2.connect(**db_params)

# Create a cursor object for executing SQL queries
cursor = connection.cursor()

# Define your SQL query
sql_query = """
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'breast_cancer';
"""

# Execute the SQL query
cursor.execute(sql_query)

# Fetch the results
results = cursor.fetchall()

# Display the results (in this example, it prints the column names)
print([row[0] for row in results if row[0] == "type"])

# Close the cursor and connection
cursor.close()
connection.close()


['type']


In [101]:
# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Create a cursor object for executing SQL queries
c = conn.cursor()


def average_columns_by_type(table_name):
    # Get the list of column names from the table
    c.execute(f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}';")
    results = c.fetchall()
    column_names = [row[0] for row in results]

    # Exclude the first column from the list
    column_names_except_first = [col for col in column_names if col != 'type']

    # Generate the SQL query dynamically with the AVG function for each column
    column_avg_expr = ', '.join([f'AVG("{col}") AS "avg_{col}"' for col in column_names_except_first])
    query = f"SELECT type, {column_avg_expr} FROM {table_name} GROUP BY type;"

    # Execute the query
    c.execute(query)
    return c.fetchall()



# Close the cursor and connection when you're done
# c.close()
# conn.close()

(average_columns_by_type('breast_cancer'))


[('basal',
  4.5246052689247565,
  7.526670340267438,
  3.6155113951687943,
  3.7538819937293892,
  5.839492037390648,
  8.878811393120596,
  3.662315048183296,
  7.02609323365351,
  4.1323499731079085,
  6.48443276564335,
  8.016006641675736,
  3.9601584679645807,
  7.668934862179891,
  6.416315807162683,
  3.83811804285543,
  8.882566727998768,
  7.195290389499494,
  6.605453092057483,
  5.175921049369986,
  7.565269048544454,
  8.872560804653215,
  7.954570041738165,
  4.208465906757869,
  6.482829314516933,
  4.5295360773688955,
  7.404063432589859,
  3.9067801818673367,
  4.666511179729897,
  4.368199737731553,
  6.934617972579049,
  3.6704706523587975,
  3.251004141932184,
  6.336524283197398,
  4.145886955931473,
  10.035671369927933,
  4.7469481030900855,
  4.985150179200712,
  8.1764496166737,
  5.505201610888158,
  8.520931216942426,
  8.466883443013268,
  8.053463801948025,
  6.521500714464835,
  2.881172953731241,
  5.120999182291001,
  5.779222621524015,
  3.04307904599144